In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from unet import Unet

model = Unet(
  image_channels=3,
)

In [4]:
model

Unet(
  (time_encoding): SinusoidalPositionalEncoding()
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
  )
  (contracting_path): ModuleList(
    (0): DownBlock(
      (time_embedding_layer): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): ReLU()
      )
      (conv1): ConvBlock(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (norm): GroupNorm(1, 128, eps=1e-05, affine=True)
        (act): ReLU()
      )
      (conv2): ConvBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (norm): GroupNorm(1, 128, eps=1e-05, affine=True)
        (act): ReLU()
      )
      (downsample): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownBlock(
      (time_embedding_layer): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): 